In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool, GCNConv
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
import random
import hashlib

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

def get_model_hash(model):
    params = []
    for k, v in model.state_dict().items():
        if "num_batches_tracked" in k:
            continue
        params.append(v.detach().cpu().float().view(-1))
    params = torch.cat(params).numpy()
    return hashlib.md5(params.tobytes()).hexdigest()

class SimpleGCN(nn.Module):
    def __init__(self, node_dim, edge_dim, global_dim, hidden_dims, dropout=0.2):
        super().__init__()
        self.node_norm = nn.BatchNorm1d(node_dim)
        if edge_dim:
            self.edge_norm = nn.BatchNorm1d(edge_dim)
        if global_dim:
            self.global_norm = nn.BatchNorm1d(global_dim)
            self.global_mlp = nn.Sequential(
                nn.Linear(global_dim, hidden_dims[-1]),
                nn.ReLU(),
                nn.Dropout(dropout)
            )
        self.convs = nn.ModuleList()
        in_dim = node_dim
        for h_dim in hidden_dims:
            self.convs.append(GCNConv(in_dim, h_dim))
            in_dim = h_dim
        self.dropout = nn.Dropout(dropout)
        self.final_dim = hidden_dims[-1] * (2 if global_dim else 1)
        self.output_mlp = nn.Sequential(
            nn.Linear(self.final_dim, self.final_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(self.final_dim // 2, 1)
        )

    def forward(self, data, return_feat=False):
        x = self.node_norm(data.x)
        if hasattr(data, 'edge_attr') and data.edge_attr is not None:
            _ = self.edge_norm(data.edge_attr)
        u = data.u if hasattr(data, 'u') else None
        if u is not None:
            u = self.global_norm(u)
        
        for conv in self.convs:
            x = F.relu(conv(x, data.edge_index))
            x = self.dropout(x)
        
        node_pool = global_mean_pool(x, data.batch)
        h = torch.cat([node_pool, self.global_mlp(u)], dim=1) if u is not None else node_pool
        out = self.output_mlp(h).squeeze()
        return (out, h) if return_feat else out

def create_data_loader(graph_data, batch_size=32, shuffle=True, seed=42):
    data_list = []
    for graph in graph_data:
        x = graph['x'].float()
        edge_index = graph['edge_index'].long()
        edge_attr = graph.get('edge_attr', None)
        if edge_attr is not None:
            edge_attr = edge_attr.float()
        u = graph.get('u', None)
        if u is not None:
            u = u.float()
        y = graph['y'].float().view(-1)
        
        data_list.append(Data(
            x=x,
            edge_index=edge_index,
            edge_attr=edge_attr,
            u=u,
            y=y
        ))
    
    if shuffle:
        generator = torch.Generator().manual_seed(seed)
        return DataLoader(
            data_list,
            batch_size=batch_size,
            shuffle=True,
            generator=generator,
            num_workers=0
        )
    else:
        return DataLoader(data_list, batch_size=batch_size, shuffle=False, num_workers=0)

def train_model(
    train_data_dir: str,
    val_data_dir: str,
    save_path: str,
    epochs=1000,
    batch_size=32,
    lr=1e-4,
    min_lr=1e-6,
    hidden_dims=[64, 64],
    dropout=0.2,
    lr_patience=10,
    es_patience=100,
    seed=42,
    no_save_epochs=200
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    if torch.cuda.is_available():
        print(f"GPU model: {torch.cuda.get_device_name(0)}")
    print(f"⚠️  No optimal R2 calculation and model saving for the first {no_save_epochs} epochs | Early stopping if no improvement for {es_patience} consecutive epochs")

    train_graph_data = torch.load(os.path.join(train_data_dir, "graph_data.pt"), map_location='cpu')
    val_graph_data = torch.load(os.path.join(val_data_dir, "graph_data.pt"), map_location='cpu')

    train_y = torch.stack([g['y'] for g in train_graph_data]).view(-1).float()
    y_mean, y_std = train_y.mean().item(), train_y.std().item() + 1e-8
    for g in train_graph_data + val_graph_data:
        g['y'] = (g['y'] - y_mean) / y_std

    train_loader = create_data_loader(train_graph_data, batch_size, True, seed=seed)
    val_loader = create_data_loader(val_graph_data, batch_size, False, seed=seed)
    val_batch = next(iter(val_loader)) if len(val_loader) > 0 else None

    sample = train_graph_data[0]
    node_dim = sample['x'].size(1)
    edge_dim = sample.get('edge_attr').size(1) if sample.get('edge_attr') is not None else 0
    global_dim = sample.get('u').size(1) if sample.get('u') is not None else 0

    model = SimpleGCN(node_dim, edge_dim, global_dim, hidden_dims, dropout).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=lr_patience, min_lr=min_lr
    )

    history = {'train_loss': [], 'val_loss': [], 'train_r2': [], 'val_r2': []}
    best_val_r2 = -float('inf')
    patience = 0
    r2_effective_thresh = 0.005
    best_r2_rel_thresh = 0.005

    print(f"\n🚀 Start training (total {epochs} epochs, seed={seed}) ===")
    print(f"R2 effective improvement standard: absolute improvement ≥{r2_effective_thresh} or relative improvement ≥{best_r2_rel_thresh*100}% of the best R2")

    for epoch in range(1, epochs + 1):
        model.train()
        epoch_loss = 0.0
        for batch in train_loader:
            batch = batch.to(device, non_blocking=True)
            pred, _ = model(batch, return_feat=True)
            loss = F.mse_loss(pred, batch.y.view(-1))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        model.eval()
        def eval_r2_loss(loader):
            ys, preds, total_loss = [], [], 0.0
            with torch.no_grad():
                for batch in loader:
                    batch = batch.to(device, non_blocking=True)
                    out = model(batch)
                    loss = F.mse_loss(out, batch.y.view(-1))
                    total_loss += loss.item() * batch.num_graphs
                    ys.append(batch.y.view(-1).cpu().numpy())
                    preds.append(out.cpu().numpy())
            avg_loss = total_loss / len(loader.dataset)
            r2 = r2_score(np.concatenate(ys), np.concatenate(preds))
            return r2, avg_loss

        train_r2, train_avg_loss = eval_r2_loss(train_loader)
        val_r2, val_avg_loss = eval_r2_loss(val_loader)
        current_lr = optimizer.param_groups[0]['lr']

        history['train_loss'].append(train_avg_loss)
        history['val_loss'].append(val_avg_loss)
        history['train_r2'].append(train_r2)
        history['val_r2'].append(val_r2)

        r2_effective = False
        log_msg = ""
        current_val_r2 = val_r2

        if epoch <= no_save_epochs:
            log_msg = f"[First {no_save_epochs} epochs] Epoch {epoch:3d} | Train R2: {train_r2:.4f} | Val R2: {current_val_r2:.4f} (not for optimal calculation)"
        else:
            rel_r2_thresh = max(r2_effective_thresh, best_val_r2 * best_r2_rel_thresh)
            
            if current_val_r2 > best_val_r2:
                r2_delta = current_val_r2 - best_val_r2
                if r2_delta >= r2_effective_thresh or r2_delta >= rel_r2_thresh:
                    r2_effective = True
                    best_val_r2 = current_val_r2
                    
                    current_hash = get_model_hash(model)
                    with torch.no_grad():
                        fixed_output = model(val_batch.to(device)).cpu().numpy() if val_batch is not None else None
                    
                    torch.save({
                        'model_state_dict': model.state_dict(),
                        'y_mean': y_mean,
                        'y_std': y_std,
                        'node_dim': node_dim,
                        'edge_dim': edge_dim,
                        'global_dim': global_dim,
                        'hidden_dims': hidden_dims,
                        'dropout': dropout,
                        'best_val_r2': best_val_r2,
                        'best_train_r2': train_r2,
                        'param_hash': current_hash,
                        'fixed_output': fixed_output,
                        'seed': seed,
                        'saved_epoch': epoch
                    }, save_path)
                    log_msg = (f"[Effective improvement] Epoch {epoch:3d} | "
                               f"Train R2: {train_r2:.4f} | Val R2 updated to {best_val_r2:.4f} (improvement {r2_delta:.4f}) | "
                               f"Hash={current_hash[:8]}... | Model saved")
                else:
                    log_msg = (f"[Minor improvement] Epoch {epoch:3d} | "
                               f"Train R2: {train_r2:.4f} | Val R2={current_val_r2:.4f} (improvement {r2_delta:.4f} < threshold) | "
                               f"Current best Val R2={best_val_r2:.4f}")
            else:
                log_msg = (f"[No improvement] Epoch {epoch:3d} | "
                           f"Train R2: {train_r2:.4f} | Val R2={current_val_r2:.4f} | "
                           f"Current best Val R2={best_val_r2:.4f}")

        if epoch > no_save_epochs:
            if r2_effective:
                patience = 0
                print(log_msg)
            else:
                patience += 1
                patience_checks = [es_patience//4, es_patience//2, int(es_patience*0.75), es_patience]
                if patience % 10 == 0 or patience in patience_checks:
                    print(f"[Early stopping counter] Epoch {epoch:3d} | No effective improvement for {patience}/{es_patience} epochs | "
                          f"Train R2: {train_r2:.4f} | Current Val R2={current_val_r2:.4f} | Best Val R2={best_val_r2:.4f}")
        else:
            if epoch % 10 == 0 or epoch == 1:
                print(log_msg)

        if epoch == no_save_epochs:
            print(f"\n[Stage switch] Epoch {epoch:3d} | First {no_save_epochs} epochs ended, start optimal R2 calculation and model saving")

        if epoch > no_save_epochs and patience >= es_patience:
            print(f"\n🛑 [Early stopping triggered] Epoch {epoch:3d} | No effective R2 improvement for {es_patience} consecutive epochs | "
                  f"Best Val R2={best_val_r2:.4f} | Corresponding Train R2={history['train_r2'][epoch - patience - 1]:.4f}")
            break

        scheduler.step(epoch_loss)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_r2'], label='Train R2', color='blue')
    plt.plot(history['val_r2'], label='Val R2', color='orange')
    plt.axhline(y=best_val_r2, color='red', linestyle='--', label=f'Best Val R2 ({best_val_r2:.4f})')
    plt.axvline(x=no_save_epochs, color='gray', linestyle=':', label=f"Optimal calculation start ({no_save_epochs})")
    if epoch > no_save_epochs:
        best_epoch = epoch - patience
        plt.axvline(x=best_epoch, color='green', linestyle='-.', label=f'Best R2 epoch ({best_epoch})')
        plt.scatter(best_epoch, best_val_r2, color='red', s=100, zorder=5)
    plt.title('Training & Validation R2')
    plt.xlabel('Epoch')
    plt.ylabel('R2 Score')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(1, 2, 2)
    plt.plot(history['train_loss'], label='Train Loss', color='blue')
    plt.plot(history['val_loss'], label='Val Loss', color='orange')
    plt.axvline(x=no_save_epochs, color='gray', linestyle=':', label=f"Optimal calculation start ({no_save_epochs})")
    plt.title('Training & Validation Loss (For reference only)')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(save_path.replace('.pt', '_curves.png'))
    plt.close()

    if os.path.exists(save_path):
        saved_info = torch.load(save_path)
        print(f"\n🎯 Training completed! Best Val R2: {best_val_r2:.4f} (Corresponding Train R2: {saved_info['best_train_r2']:.4f}), "
              f"Model saved at {save_path} (Saved epoch: {saved_info['saved_epoch']})")
    else:
        print(f"\n🎯 Training completed! No model saved (Early stopped at epoch {epoch}, not exceeding {no_save_epochs} epochs or no effective improvement)")

    return best_val_r2, save_path

if __name__ == "__main__":
    # Directory of training graph data
    train_data_dir = "TRAIN_DATA_DIR_PATH"
    # Directory of validation graph data
    val_data_dir = "VAL_DATA_DIR_PATH"
    # Directory to save trained models
    save_dir = "MODEL_SAVE_DIR_PATH"

    seeds = [0, 42, 100, 2025]
    os.makedirs(save_dir, exist_ok=True)
    all_results = []

    for seed in seeds:
        print(f"\n===== Current random seed: {seed} =====")
        set_seed(seed)
        out_path = os.path.join(save_dir, f"gcn_best_seed({seed})_128_128_0.1.pt")
        best_val_r2, _ = train_model(
            train_data_dir=train_data_dir,
            val_data_dir=val_data_dir,
            save_path=out_path,
            epochs=1500,
            batch_size=32,
            lr=1e-3,
            min_lr=1e-4,
            hidden_dims=[128, 128],
            dropout=0.1,
            lr_patience=20,
            es_patience=100,
            seed=seed,
            no_save_epochs=200
        )
        if os.path.exists(out_path):
            best_train_r2 = torch.load(out_path)['best_train_r2']
        else:
            best_train_r2 = -float('inf')
        all_results.append((seed, best_val_r2, best_train_r2))
        print(f"Seed {seed} → Best Val R2: {best_val_r2:.4f} | Corresponding Train R2: {best_train_r2:.4f}")

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    print("\n===== Multi-seed training results summary =====")
    print(f"{'Seed':<6} | {'Best Val R2':<12} | {'Corresponding Train R2':<12}")
    print("-" * 40)
    for s, val_r2, train_r2 in all_results:
        print(f"{s:<6} | {val_r2:.4f}        | {train_r2:.4f}")
    val_r2s = [r[1] for r in all_results]
    train_r2s = [r[2] for r in all_results if r[2] != -float('inf')]
    print(f"\nAverage Val R2 = {np.mean(val_r2s):.4f} ± {np.std(val_r2s):.4f}")
    if train_r2s:
        print(f"Average Train R2 = {np.mean(train_r2s):.4f} ± {np.std(train_r2s):.4f}")
